image encryption and decryption


In [1]:
import numpy as np
from PIL import Image
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import Model
from google.colab import files

# Upload image
uploaded = files.upload()
image_path = list(uploaded.keys())[0]  # Get uploaded filename

# Parameters for the 3D hyperchaotic map
a1, a2, a3 = 0.05, 0.25, 0.12
b1, b2 = 4, 1.2
c = 2.15
x0, y0, z0 = 0.1, 0.1, 0.1  # Initial conditions

# Global vars to store for decryption
global_shuffled_pixels = None
global_height, global_width, global_channels = None, None, None

def generate_chaotic_sequence(length):
    x, y, z = x0, y0, z0
    X, Y, Z = [], [], []
    for _ in range(length):
        X.append(x)
        Y.append(y)
        Z.append(z)
        x_next = (x + a1 * x + a2 * y + a3 * y**2) % 1
        y_next = (y + b1 - b2 * z) % 1
        z_next = (z + c * x) % 1
        x, y, z = x_next, y_next, z_next
    return np.array(X), np.array(Y), np.array(Z)

def extract_features(img_array):
    """
    Extract features using pre-trained VGG16.
    """
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    model = Model(inputs=base_model.input, outputs=base_model.get_layer('block5_pool').output)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    features = model.predict(img_array, verbose=0)
    return features.flatten()

def encrypt_image(image_path, output_encrypted_path):
    """
    Encrypts the image using VGG16 features and chaotic sequence.
    """
    global global_shuffled_pixels, global_height, global_width, global_channels

    # Load image and force RGB
    img = Image.open(image_path).convert("RGB")
    img_array = np.array(img)
    height, width, channels = img_array.shape
    flat_img = img_array.reshape(-1, channels)
    total_pixels = height * width

    # Resize for VGG16
    img_resized = img.resize((224, 224))
    img_array_resized = np.array(img_resized)
    if img_array_resized.ndim == 2:
        img_array_resized = np.stack([img_array_resized]*3, axis=-1)

    # Extract VGG16 features (not used in encryption here, but included as per your request)
    print(" Extracting VGG16 features...")
    vgg_features = extract_features(img_array_resized)
    print(f"VGG16 features shape: {vgg_features.shape}")

    # Chaotic sequence
    X, _, _ = generate_chaotic_sequence(total_pixels)
    perm_indices = np.argsort(X)
    shuffled_pixels = flat_img[perm_indices]

    # Store for decryption
    global_shuffled_pixels = shuffled_pixels.copy()
    global_height, global_width, global_channels = height, width, channels

    # Save encrypted image
    encrypted_img_array = shuffled_pixels.reshape(height, width, channels)
    encrypted_img = Image.fromarray(encrypted_img_array.astype('uint8'))
    encrypted_img.save(output_encrypted_path)
    print(f" Encrypted image saved to: {output_encrypted_path}")

def decrypt_image(output_decrypted_path):
    """
    Decrypt the image using the inverse chaotic permutation.
    """
    global global_shuffled_pixels, global_height, global_width, global_channels

    if global_shuffled_pixels is None:
        print(" Error: No encrypted pixels stored.")
        return

    height, width, channels = global_height, global_width, global_channels
    total_pixels = height * width

    X, _, _ = generate_chaotic_sequence(total_pixels)
    perm_indices = np.argsort(X)
    reverse_perm_indices = np.argsort(perm_indices)

    decrypted_pixels = global_shuffled_pixels[reverse_perm_indices]
    decrypted_img_array = decrypted_pixels.reshape(height, width, channels)

    decrypted_img = Image.fromarray(decrypted_img_array.astype('uint8'))
    decrypted_img.save(output_decrypted_path)
    print(f" Decrypted image saved to: {output_decrypted_path}")

# Paths
encrypted_path = "/content/encrypted_image.jpg"
decrypted_path = "/content/decrypted_image.jpg"

# Encrypt and Decrypt
encrypt_image(image_path, encrypted_path)
decrypt_image(decrypted_path)

# Download images
files.download(encrypted_path)
files.download(decrypted_path)


Saving image.jpg to image (1).jpg
 Extracting VGG16 features...
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
VGG16 features shape: (25088,)
 Encrypted image saved to: /content/encrypted_image.jpg
 Decrypted image saved to: /content/decrypted_image.jpg


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>